In [134]:
import pypsa
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline



In [135]:
# power is purchased from generators to satisfy loads
# p_nom is consumed until load is satisfied, the final generation needed becomes the marginal cost
# p_nom_extendable allows the marginal cost to be purchased by adding generators
# the marginal price is available on the bus
# a price sensitive load can be created with a Generator by setting p_min_pu = -1, p_max_pu = 0, and a marginal_cost

In [136]:
marginal_costs = {"Wind": 0, "Hydro": 0, "Coal": 30, "Gas": 60, "Oil": 80}

# power plant capacities (nominal powers in MW) in each country (not necessarily realistic)
power_plant_p_nom = {
    "South Africa": {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000},
    "Mozambique": {
        "Hydro": 1200,
    },
    "Swaziland": {
        "Hydro": 600,
    },
}

# transmission capacities in MW (not necessarily realistic)
transmission = {
    "South Africa": {"Mozambique": 500, "Swaziland": 250},
    "Mozambique": {"Swaziland": 100},
}

# country electrical loads in MW (not necessarily realistic)
loads = {"South Africa": 42000, "Mozambique": 650, "Swaziland": 250}


country = "South Africa"

network = pypsa.Network()

# snapshots labelled by [0,1,2,3]
network.set_snapshots(range(4))

network.add("Bus", country)

# p_max_pu is variable for wind
for tech in power_plant_p_nom[country]:
    network.add(
        "Generator",
        f"{country} {tech}",
        bus=country,
        p_nom=power_plant_p_nom[country][tech],
        marginal_cost=marginal_costs[tech],
        p_max_pu=([0.3, 0.6, 0.4, 0.5] if tech == "Wind" else 1),
    )

# load which varies over the snapshots
network.add(
    "Load",
    f"{country} load",
    bus=country,
    p_set=loads[country] + np.array([0, 1000, 3000, 4000]),
)

# storage unit to do price arbitrage
network.add(
    "StorageUnit",
    f"{country} pumped hydro",
    bus=country,
    p_nom=1000,
    max_hours=6,  # energy storage in terms of hours at full power
    state_of_charge_set=pd.Series([np.nan, np.nan, 100, np.nan])
)

In [137]:

# import sys
# from pathlib import Path
# sys.path.append(str(Path.cwd().parent))
# from gridsim_backend.app.network import get_single_node_network

# n = get_single_node_network()

# n.plot()



In [138]:
import logging

logging.basicConfig(level=logging.DEBUG)

pypsa.optimization.optimize.logger.setLevel(logging.DEBUG)

network.optimize()
# print('objective', network.objective)
# print('statistics', network.statistics())
network.storage_units_t.p

Index(['South Africa'], dtype='object', name='Bus')
Index(['South Africa'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /private/var/folders/0d/q51cf8693zj50wd4hy84k0zh0000gn/T/highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 28 primals, 65 duals
Objective: 6.06e+06
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-state_of_charge_set, StorageUnit-energy_balance were not assigned to the network.


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e+01, 8e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 5e+04]
Presolving model
8 rows, 23 cols, 33 nonzeros  0s
1 rows, 4 cols, 3 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-65); columns 0(-28); elements 0(-96) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  6.0640000000e+06
HiGHS run time      :          0.00
Writing the solution to /private/var/folders/0d/q51cf8693zj50wd4hy84k0zh0000gn/T/linopy-solve-tf51msrs.sol


StorageUnit,South Africa pumped hydro
snapshot,
0,-100.0
1,-1000.0
2,1000.0
3,100.0


In [139]:
df = pd.DataFrame(
    {attr: n.stores_t[attr]["battery storage"] for attr in ["p", "e"]}
)
df.plot(grid=True, figsize=(20, 10))
plt.legend(labels=["Energy output", "State of charge"])
plt.tight_layout()

KeyError: 'battery storage'

In [24]:
n.stores_t

{'e_min_pu': Empty DataFrame
 Columns: []
 Index: [2025-03-01 00:00:00, 2025-03-01 00:05:00, 2025-03-01 00:10:00, 2025-03-01 00:15:00, 2025-03-01 00:20:00, 2025-03-01 00:25:00, 2025-03-01 00:30:00, 2025-03-01 00:35:00, 2025-03-01 00:40:00, 2025-03-01 00:45:00, 2025-03-01 00:50:00, 2025-03-01 00:55:00, 2025-03-01 01:00:00, 2025-03-01 01:05:00, 2025-03-01 01:10:00, 2025-03-01 01:15:00, 2025-03-01 01:20:00, 2025-03-01 01:25:00, 2025-03-01 01:30:00, 2025-03-01 01:35:00, 2025-03-01 01:40:00, 2025-03-01 01:45:00, 2025-03-01 01:50:00, 2025-03-01 01:55:00, 2025-03-01 02:00:00, 2025-03-01 02:05:00, 2025-03-01 02:10:00, 2025-03-01 02:15:00, 2025-03-01 02:20:00, 2025-03-01 02:25:00, 2025-03-01 02:30:00, 2025-03-01 02:35:00, 2025-03-01 02:40:00, 2025-03-01 02:45:00, 2025-03-01 02:50:00, 2025-03-01 02:55:00, 2025-03-01 03:00:00, 2025-03-01 03:05:00, 2025-03-01 03:10:00, 2025-03-01 03:15:00, 2025-03-01 03:20:00, 2025-03-01 03:25:00, 2025-03-01 03:30:00, 2025-03-01 03:35:00, 2025-03-01 03:40:00, 2025